In [1]:
import os
import glob
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import timm
from tqdm import tqdm

import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F


In [2]:
SIZE=48
BATCH_SIZE=512

In [3]:
class UTKFaceDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.files = glob.glob(os.path.join(directory, '*.jpg'))
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img_name = self.files[idx]
        image = Image.open(img_name)
        filename = img_name.split('/')[-1]
        sex = int(filename.split('_')[1])  # Предполагается, что имя файла начинается с возраста

        if self.transform:
            image = self.transform(image)

        return image, sex

In [4]:
transform = transforms.Compose([
    transforms.Resize((SIZE, SIZE)),
    transforms.ToTensor(),
])

dataset = UTKFaceDataset(directory='../data/UTKFace_48', transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
class SexEstimatorModel(nn.Module):
    def __init__(self):
        super(SexEstimatorModel, self).__init__()
        self.base_model = timm.create_model('efficientnetv2_rw_s', pretrained=True)
        self.base_model.classifier = nn.Linear(self.base_model.classifier.in_features, 1)
        
    def forward(self, x):
        x = self.base_model(x)
        return torch.sigmoid(x).squeeze()  # Используйте сигмоиду для бинарной классификации


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = SexEstimatorModel().to(device)
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
for epoch in range(10):  # проход по датасету несколько раз
    running_loss = 0.0
    for i, data in tqdm(enumerate(train_loader)):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.view(-1), labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, loss: {running_loss / len(train_loader)}')
print('Finished Training')

36it [00:18,  1.93it/s]


Epoch 1, loss: 0.21437676747639975


36it [00:14,  2.44it/s]


Epoch 2, loss: 0.10944789296223058


36it [00:14,  2.42it/s]


...

Epoch 8, loss: 0.07071137200627062


36it [00:14,  2.45it/s]


Epoch 9, loss: 0.07107524718675348


36it [00:14,  2.45it/s]

Epoch 10, loss: 0.09042107603616184
Finished Training


In [8]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Функция для вычисления предсказаний
def get_predictions(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            preds = outputs.round()  # Округление до 0 или 1
            all_preds.extend(preds.tolist())
            all_labels.extend(labels.tolist())
    return all_labels, all_preds

# Получение предсказаний на валидационном наборе
val_labels, val_preds = get_predictions(model, test_loader)

# Вычисление метрик
precision = precision_score(val_labels, val_preds)
recall = recall_score(val_labels, val_preds)
f1 = f1_score(val_labels, val_preds)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Precision: 0.88259526261586
Recall: 0.7894979272224781
F1 Score: 0.8334548991004133


In [9]:
torch.save(model, '../../sex_model_torch.pth')
torch.save(model.state_dict(), '../../sex_model_weights.pth')

In [15]:
# model = SexEstimatorModel()  # Создайте экземпляр вашей модели
# model.load_state_dict(torch.load('../../sex_model_torch.pth'))

model = torch.load('../../sex_model_torch.pth')

model.eval()  # Переведите модель в режим оценки

# Загрузите изображение
image_path = '../data/face_recognition_images/person2.1.jpg'
image = Image.open(image_path)

# Примените преобразования к изображению
image = transform(image)
image = image.to(device)
image = image.unsqueeze(0)  # Добавьте дополнительное измерение, так как модель ожидает пакет изображений

# Сделайте предсказание
with torch.no_grad():
    output = model(image)
    predicted_sex = output.round().item()  # Получите предсказанный возраст как число

print(f'Predicted Sex: {predicted_sex}')

Predicted Sex: 0.0
